In [ ]:
import pandas as pd
data = pd.read_csv(r'C:\Users\adima\Downloads\cv-corpus-19.0-delta-2024-09-13-en\cv-corpus-19.0-delta-2024-09-13\en\other.tsv', sep='\t')
data



In [ ]:
# Filter rows where gender or age are missing
data = data[data['gender'].isin(['female_feminine', 'male_masculine'])]
data = data.dropna(subset=['gender', 'age'])
data=data.drop(["client_id","sentence_id","sentence","sentence_domain","up_votes","down_votes","accents","variant","locale","segment"],axis=1)
data

In [ ]:
data['age'].value_counts()

In [ ]:
print(data['gender'].value_counts())

In [ ]:
data['gender'] = data['gender'].astype('category')
data

In [ ]:
data['age'] = data['age'].replace({
    'teens': 'teens_twenties', 
    'twenties': 'teens_twenties',
    'fifties': 'fifties_sixties', 
    'sixties': 'fifties_sixties'
})
data['age'].value_counts()
# Keep only rows where 'path', 'age', and 'gender' are non-null
data = data.dropna(subset=['path', 'age', 'gender'])

# Check the result
data.info()


In [ ]:
import librosa
import numpy as np
import os 

def extract_features(audio_path):
    # Load audio file
    audio, sr = librosa.load(audio_path)
    
    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    
    # Average MFCC over time to get a feature vector
    mfcc = np.mean(mfcc.T, axis=0)
    return mfcc


In [ ]:
audio_dir = r'C:\Users\adima\Downloads\cv-corpus-19.0-delta-2024-09-13-en\cv-corpus-19.0-delta-2024-09-13\en\clips'
features = []
for index, row in data.iterrows():
    # Get the file name from the 'path' column
    file_name = row['path']
    
    # Combine directory path with file name to get the full path
    audio_path = os.path.join(audio_dir, file_name)
    features.append(extract_features(audio_path))
    
  
# Convert the list of features to a NumPy array
features = np.array(features)



In [ ]:
data['age']=data['age'].replace({
                             'teens_twenties':0,
                             'thirties':1,
                             'fourties':2,
                             'fifties_sixties':3})
data['gender']=data['gender'].replace({
                             'male_masculine':0,
                             'female_feminine':1,})
data['gender'].value_counts()

In [ ]:
#data=data.drop(["client_id","sentence_id","sentence","sentence_domain","up_votes","down_votes","accents","variant","locale","segment"],axis=1)
data.info()

In [ ]:
X = features  # Audio features
y_gender = data['gender']  
y_age = data['age']  
data.info()

In [ ]:
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_gender_train1, y_gender_test1 = train_test_split(X, y_gender, test_size=0.2 ,random_state=42)
X_train, X_test, y_age_train, y_age_test = train_test_split(X, y_age, test_size=0.2 ,random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


gender_model = RandomForestClassifier(class_weight='balanced')
gender_model.fit(X_train1, y_gender_train1)

y_gender_pred1 = gender_model.predict(X_test1)
print("Gender Classification Accuracy:", accuracy_score(y_gender_test1, y_gender_pred1))

cv_scores = cross_val_score(age_model, X_train1, y_gender_train1, cv=5, scoring='accuracy')
print("Cross-validation Accuracy Scores:", cv_scores)
print("Mean Cross-validation Accuracy:", np.mean(cv_scores))
print("Standard Deviation of Cross-validation Accuracy:", np.std(cv_scores))


In [ ]:
age_model=RandomForestClassifier()
age_model.fit(X_train,y_age_train)

y_age_pred = age_model.predict(X_test)
print("Age Classification Accuracy:", accuracy_score(y_age_test, y_age_pred))

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np


# Perform cross-validation and calculate accuracy scores
cv_scores = cross_val_score(age_model, X_train, y_age_train, cv=5, scoring='accuracy')

# Output the cross-validation results
print("Cross-validation Accuracy Scores:", cv_scores)
print("Mean Cross-validation Accuracy:", np.mean(cv_scores))
print("Standard Deviation of Cross-validation Accuracy:", np.std(cv_scores))


Testing IRL

In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write

# Set sample rate and duration
sample_rate = 16000  # Match your model's sample rate
duration = 5  # Seconds

# Record audio
print("Recording...")
audio_data = sd.rec(int(sample_rate * duration), samplerate=sample_rate, channels=1)
sd.wait()  # Wait until recording is finished
print("Recording complete.")

# Save to file
write("your_voice_sample.wav", sample_rate, audio_data)


In [ ]:
import librosa
import numpy as np

def preprocess_audio(file_path):
    # Load the audio file
    y, sr = librosa.load(file_path, sr=16000)  # Ensure correct sample rate

    # Extract MFCCs or spectrogram (use the same features as used in training)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs = np.mean(mfccs.T, axis=0)  # Average MFCCs over time
    return mfccs

audio_features = preprocess_audio("your_voice_sample.wav")


In [ ]:

# Reshape audio_features if necessary
audio_features = audio_features.reshape(1, -1)  # Make it a 2D array for model input
# Predict gender
gender_prediction = gender_model.predict(audio_features)
age_prediction = age_model.predict(audio_features)
# Interpret the output
gender = "Male" if gender_prediction[0] == 0 else "Female"  # or use actual labels if your model uses them
# Define a mapping from numeric prediction to age category
age_mapping = {
    0: "teens_twenties",
    1: "thirties",
    2: "fourties",
    3: "fifties_sixties"
}

# Get the predicted age category based on the prediction output
age_prediction_value = age_prediction[0]
age = age_mapping.get(age_prediction_value, "Unknown")  # Default to "Unknown" if the value is not in the mapping

print(f"Predicted Age Category: {age}")

print(f"Predicted Gender: {gender}")


In [ ]:
'''import pickle

# Save the model
with open('age_model.pkl', 'wb') as file:
    pickle.dump(age_model, file)
'''

In [ ]:
'''with open('gender_model.pkl', 'wb') as file:
    pickle.dump(age_model, file)'''